In [ ]:
from langchain.vectorstores import DocArrayInMemorySearch

In [ ]:
%pip install langchain-experimental

In [2]:

from langchain_community.chat_models import ChatOllama
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_experimental.tools import PythonREPLTool


from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

instructions = """You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
"""
base_prompt = hub.pull("hwchase17/react")
prompt = base_prompt.partial(instructions=instructions)


llm = ChatOllama(model="mistral:instruct") 



search = DuckDuckGoSearchRun(name="Search")

    
tools = [search]
agent = create_react_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

agent_executor.invoke({"input": "What is the 10th fibonacci number?"})



> Entering new AgentExecutor chain...


KeyboardInterrupt: 

In [3]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], partial_variables={'instructions': 'You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an error, debug your code and try again.\nOnly use the output of your code to answer the question. \nYou might know the answer without running any code, but you should still run the code to get the answer.\nIf it does not seem like you can write code to answer the question, just return "I don\'t know" as the answer.\n'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this 

In [ ]:

import requests

BASE_URL = "http://192.168.86.29:11434"

def list_models():
    response = requests.get(f"{BASE_URL}/api/tags")
    if response.status_code == 200:
        models = response.json().get("models", [])
        return [model["name"] for model in models]
    else:
        print(f"Error: {response.status_code}")
        return []
    
list_models()

In [ ]:
from typing import List
import pandas as pd
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_community.chat_models.ollama import ChatOllama
from rich import print as rprint
from prompts import presidents
import requests
from prompt_toolkit import prompt
from prompt_toolkit.completion import WordCompleter


BASE_URL = "http://192.168.86.29:11434"

def list_models():
    response = requests.get(f"{BASE_URL}/api/tags")
    if response.status_code == 200:
        models = response.json().get("models", [])
        return [model["name"] for model in models]
    else:
        print(f"Error: {response.status_code}")
        return []



class Term(BaseModel):
    start: str = Field(..., description="The start year of the president's term.")
    end: str = Field(..., description="The end year of the president's term .")


class President(BaseModel):
    name: str = Field(..., description="The name of the president.")
    birth_year: int = Field(..., description="The birth year of the president.")
    death_year: int = Field(..., description="The death year of the president.")
    party: str = Field(..., description="The political party of the president.")
    term: Term = Field(..., description="The term of the president.")
    interesting_fact: str = Field(..., description="An interesting fact about the president.")
    popular_vote: int = Field(..., description="The popular vote of the president.")



def get_presidents(model_name: str):
    model = ChatOllama(model=model_name ,base_url=BASE_URL)
    # query a random president from presidents list
    query = pd.DataFrame(presidents).sample(1).to_string(index=False, header=False)
    rprint(f"US PRESIDENT CHOSEN: {query}")

    parser = JsonOutputParser(pydantic_object=President)

    prompt = PromptTemplate(
        template="Answer the user query.\n{format_instructions}\n{query}\n",
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )


    chain = prompt | model | parser


    output = chain.invoke({"query": query})


    rprint(output)

    return output

#model_name_list = pd.DataFrame(list_models())
## need to remove all-minilm:latest from the dataframe
#model_name_list = model_name_list[model_name_list[0] != "all-minilm:latest"][0].to_list()
model_name_list = [    'dolphin-mistral:latest',
    'dolphin-phi:latest',
    'gemma:7b',
    'gemma:instruct',
    'llama2:latest',
    'llava:latest',
    'mistral:instruct',
    'mistral:latest',
    'stablelm2:latest',
    'tinydolphin:latest',
    'zephyr:latest'
]
model_completer = WordCompleter(model_name_list, ignore_case=True)

model_name = prompt("Select a model: ", completer=model_completer)

rprint(f"TESTING: {model_name}")
try:
    get_presidents(model_name)
    rprint("\n\n\n")
except Exception as e:
    rprint(f"Error: {e}")

In [ ]:
from langchain_community.document_loaders import AsyncHtmlLoader

urls = ["https://python.langchain.com/docs/"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()

In [ ]:
docs

In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage
from langchain_community.chat_models import ChatOpenAI
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


chat = ChatOpenAI(model='LLaMA_CPP',
                    temperature=0,
                    openai_api_key="sk-no-key-required", 
                    base_url='http://localhost:8080/v1',
                    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))  






In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI




search = DuckDuckGoSearchRun()


template = """turn the following user input into a search query for a search engine:

{input}"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(base_url='http://localhost:8080/v1',
                   model='LLaMA_CPP')

chain = prompt | model | StrOutputParser() | search


chain.invoke({"input": "Current news openai"})

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="LLaMA_CPP",
                    openai_api_key="sk-no-key-required", 
                    base_url='http://localhost:8080/v1')
output_parser = StrOutputParser()

chain = prompt | model | output_parser

print(chain)



chain.invoke({"topic": "ice cream"})

In [ ]:
%pip install rich --quiet

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI



output_parser = StrOutputParser()


# 
def search(query):
    search = DuckDuckGoSearchRun()
    return search.invoke({"query": query})
    



def search_results():
    search = DuckDuckGoSearchResults()
    pass






In [ ]:
from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()
item = search.run("Obama's first name?")

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role": "user", "content": "Who won the world series in 2020?"}
  ]
)
print(response.choices[0].message.content)

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',

    # required but ignored
    api_key='ollama',
)

chat_completion = client.chat.completions.create(
    response_format={ "type": "json_object" },
    messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role": "user", "content": "Provide me a list of 5 top tech companies that are listed on US stock exchanges. Include their CEO's name and stock ticker and one interesting fact about each company."}
    ],
    model='no-censor-dolph-mistral:latest',
)

In [ ]:
json_string = chat_completion.choices[0].message.content

In [ ]:
import json

json_object = json.loads(json_string)
json_object

In [ ]:
import pandas as pd


df = pd.json_normalize(json_object['companies'])

In [ ]:
df

In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(model="mistral")

llm.invoke("Tell me a joke")

In [ ]:
from pandasai import SmartDataframe
from langchain_community.llms import Ollama
import pandas as pd


langchain_llm = Ollama(model="mistral")
df = SmartDataframe(df, config={"llm": langchain_llm})
df.chat('who is the ceo of fb based on the data?')


In [ ]:
from pandasai import SmartDataframe
import pandas as pd
from langchain_community.llms import Ollama


langchain_llm = Ollama(model="mistral")

df = SmartDataframe(df, config={"llm": langchain_llm})
df.chat('Tell me about this dataset?')


In [ ]:
agent.chat('And what is the GDP of each of these countries?')